In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import sys

sys.path.append('./helpers_models/')
sys.path.append('./data_visualization_and_augmentations/')
sys.path.append('../torch_videovision/')
sys.path.append('./important_csvs/')

from helpers_resnet import *



In [12]:
resnet = torchvision.models.resnet50(pretrained=True)
adaptive_pooling = AdaptiveConcatPool2d()
head = Head()
resnet.avgpool = adaptive_pooling
resnet.fc = head

os.environ['CUDA_VISIBLE_DEVICES']='0,1,2'

resnet = resnet.cuda()

for param in resnet.parameters():
    param.requires_grad = False
    
for param in resnet.avgpool.parameters():
    param.requires_grad = True
    
for param in resnet.fc.parameters():
    param.requires_grad = True

resnet = nn.DataParallel(resnet)
check_freeze(resnet.module)

conv1 False
bn1 False
relu True
maxpool True
layer1 False
layer2 False
layer3 False
layer4 False
avgpool True
fc True


In [3]:
from helpers_thresholds import *

In [4]:
plt.rcParams['figure.figsize'] = (12,6)
font = {'family' : 'DejaVu Sans',  'weight' : 'normal',  'size'  : 40}
plt.rc('font', **font)

In [6]:
tensor_transform = get_tensor_transform('ImageNet', False)
train_spat_transform = get_spatial_transform(2)
train_temp_transform = get_temporal_transform()
valid_spat_transform = get_spatial_transform(0)
valid_temp_transform = va.TemporalFit(size=16)

bs = 50
root_dir = '/media/scratch/astamoulakatos/nsea_video_jpegs/'
df = pd.read_csv('./small_dataset_csvs/events_with_number_of_frames_stratified.csv')
# df_train = get_df(df, 20, True, False, False)
# class_image_paths, end_idx, idx_label= get_indices(df_train, root_dir)
# train_loader = get_loader(1, 128, end_idx, class_image_paths, train_temp_transform, train_spat_transform, tensor_transform, False, True, True, 1)
df_valid = get_df(df, 20, False, True, False)
class_image_paths, end_idx, idx_label = get_indices(df_valid, root_dir)
valid_loader, valid_dataset = get_loader(1, bs, end_idx, class_image_paths, valid_temp_transform, valid_spat_transform, tensor_transform, False, True, True, 1)
df_test = get_df(df, 20, False, False, True)
class_image_paths, end_idx, idx_label = get_indices(df_test, root_dir)
test_loader, test_dataset = get_loader(1, bs, end_idx, class_image_paths, valid_temp_transform, valid_spat_transform, tensor_transform, False, True, True, 1)

torch.cuda.empty_cache()

In [13]:
load = True
if load:
    checkpoint = torch.load('/media/scratch/astamoulakatos/saved-resnet-models/fifth-small-aug-final/best-checkpoint-010epoch.pth')
    resnet.load_state_dict(checkpoint['model_state_dict'])
    print('loading pretrained freezed model!')

loading pretrained freezed model!


In [14]:
device = torch.device('cuda')

In [7]:
inputs, labels = next(iter(valid_loader))

In [9]:
labels

tensor([[0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [1., 0., 0., 0., 1.],
        [1., 0., 0., 0., 1.],
        [0., 1., 0., 0., 0.],
        [1., 0., 0., 0., 1.],
        [1., 0., 0., 0., 1.],
        [0., 1., 0., 0., 0.],
        [1., 0., 0., 0., 1.],
        [1., 0., 0., 0., 1.],
        [0., 1., 0., 0., 0.],
        [1., 0., 0., 1., 0.],
        [0., 1., 0., 0., 0.],
        [1., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [1., 0., 1., 0., 0.],
        [1., 0., 1., 0., 0.],
        [1., 0., 0., 0., 1.],
        [0., 1., 0., 0., 0.],
        [1., 0., 0., 0., 1.],
        [0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [1., 0., 0., 0., 1.],
        [0., 1., 0., 0., 0.],
        [1., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0

In [10]:
from more_losses import *

In [29]:
from warp_lsep import *

In [68]:
loss_warp_1 = loss_warp(outputs.cuda(), labels_smo.cuda())

In [69]:
loss_warp_1

tensor(0., grad_fn=<WARPBackward>)

In [66]:
loss_lsep_1 = loss_lsep(torch.sigmoid(outputs.cuda()), labels_smo.cuda())

In [67]:
loss_lsep_1

tensor([0.], grad_fn=<LSEPBackward>)

In [15]:
resnet = resnet.eval()

In [16]:
outputs, _ = resnet(inputs)

In [151]:
loss = lsep_loss_stable(outputs.cuda(), labels_smo.cuda())

In [152]:
loss

tensor(4.5562, device='cuda:0', grad_fn=<MeanBackward0>)

In [154]:
labels_smo

tensor([[0.0500, 0.9500, 0.0500, 0.0500, 0.0500],
        [0.0500, 0.9500, 0.0500, 0.0500, 0.0500],
        [0.0500, 0.9500, 0.0500, 0.0500, 0.0500],
        [0.0500, 0.9500, 0.0500, 0.0500, 0.0500],
        [0.9500, 0.0500, 0.0500, 0.0500, 0.9500],
        [0.9500, 0.0500, 0.0500, 0.0500, 0.9500],
        [0.0500, 0.9500, 0.0500, 0.0500, 0.0500],
        [0.9500, 0.0500, 0.0500, 0.0500, 0.9500],
        [0.9500, 0.0500, 0.0500, 0.0500, 0.9500],
        [0.0500, 0.9500, 0.0500, 0.0500, 0.0500],
        [0.9500, 0.0500, 0.0500, 0.0500, 0.9500],
        [0.9500, 0.0500, 0.0500, 0.0500, 0.9500],
        [0.0500, 0.9500, 0.0500, 0.0500, 0.0500],
        [0.9500, 0.0500, 0.0500, 0.9500, 0.0500],
        [0.0500, 0.9500, 0.0500, 0.0500, 0.0500],
        [0.9500, 0.0500, 0.9500, 0.0500, 0.0500],
        [0.0500, 0.9500, 0.0500, 0.0500, 0.0500],
        [0.9500, 0.0500, 0.0500, 0.0500, 0.0500],
        [0.9500, 0.0500, 0.0500, 0.0500, 0.0500],
        [0.0500, 0.9500, 0.0500, 0.0500, 0.0500],


In [147]:
loss2 = lsep_loss(outputs.cuda(), labels_smo.cuda())

In [148]:
loss2

tensor(0.6654, device='cuda:0', grad_fn=<MeanBackward0>)

In [56]:
loss_bce_1 = binary_cross_entropy(outputs.cuda(), labels_smo.cuda())

In [57]:
loss_bce_1

tensor(0.4576, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>)

In [73]:
loss_focal_1 = focal_loss(outputs.cuda(), labels_smo.cuda())

In [78]:
loss_focal_1

tensor(0.0836, device='cuda:0', grad_fn=<MeanBackward0>)

In [77]:
from focal_loss_2 import *

In [90]:
pos_wei = torch.tensor([1, 1, 3, 3, 1])
pos_wei = pos_wei.cuda()
#criterion = nn.BCEWithLogitsLoss(pos_weight = pos_wei)
criterion = FocalLoss2d(weight=pos_wei,reduction='mean',balance_param=1)

In [95]:
loss_focal_2 = criterion(outputs.cuda(), labels_smo.cuda())

In [96]:
loss_focal_2

tensor(0.1650, device='cuda:0', grad_fn=<MulBackward0>)

In [108]:
pos_wei = torch.tensor([1, 1, 3, 3, 1])
pos_wei = pos_wei.cuda()
criterion2 = nn.BCEWithLogitsLoss(pos_weight = pos_wei)

In [109]:
loss_bce_2 = criterion2(outputs.cuda(), labels_smo.cuda())

In [110]:
loss_bce_2

tensor(0.6791, device='cuda:0', grad_fn=<BinaryCrossEntropyWithLogitsBackward>)

In [111]:
differences = outputs.unsqueeze(1) - outputs.unsqueeze(2)

In [112]:
differences

tensor([[[  0.0000,   5.7382,  -1.4639,  -0.5500,  -0.6794],
         [ -5.7382,   0.0000,  -7.2021,  -6.2882,  -6.4176],
         [  1.4639,   7.2021,   0.0000,   0.9139,   0.7845],
         [  0.5500,   6.2882,  -0.9139,   0.0000,  -0.1294],
         [  0.6794,   6.4176,  -0.7845,   0.1294,   0.0000]],

        [[  0.0000,   9.0546,   1.2798,   0.6682,  -1.0879],
         [ -9.0546,   0.0000,  -7.7748,  -8.3865, -10.1425],
         [ -1.2798,   7.7748,   0.0000,  -0.6117,  -2.3677],
         [ -0.6682,   8.3865,   0.6117,   0.0000,  -1.7560],
         [  1.0879,  10.1425,   2.3677,   1.7560,   0.0000]],

        [[  0.0000,   9.6342,   0.4911,  -0.8545,  -0.1156],
         [ -9.6342,   0.0000,  -9.1432, -10.4887,  -9.7499],
         [ -0.4911,   9.1432,   0.0000,  -1.3456,  -0.6067],
         [  0.8545,  10.4887,   1.3456,   0.0000,   0.7389],
         [  0.1156,   9.7499,   0.6067,  -0.7389,   0.0000]],

        ...,

        [[  0.0000,   2.7682,  -1.7709,  -1.0570,  -3.6197],
    

In [115]:
where_different = (labels.unsqueeze(1) < labels.unsqueeze(2)).float()

In [119]:
labels.shape

torch.Size([50, 5])

In [127]:
labels_un = labels.unsqueeze(2); labels_un.shape

torch.Size([50, 5, 1])

In [128]:
labels_un

tensor([[[0.],
         [1.],
         [0.],
         [0.],
         [0.]],

        [[0.],
         [1.],
         [0.],
         [0.],
         [0.]],

        [[0.],
         [1.],
         [0.],
         [0.],
         [0.]],

        [[0.],
         [1.],
         [0.],
         [0.],
         [0.]],

        [[1.],
         [0.],
         [0.],
         [0.],
         [1.]],

        [[1.],
         [0.],
         [0.],
         [0.],
         [1.]],

        [[0.],
         [1.],
         [0.],
         [0.],
         [0.]],

        [[1.],
         [0.],
         [0.],
         [0.],
         [1.]],

        [[1.],
         [0.],
         [0.],
         [0.],
         [1.]],

        [[0.],
         [1.],
         [0.],
         [0.],
         [0.]],

        [[1.],
         [0.],
         [0.],
         [0.],
         [1.]],

        [[1.],
         [0.],
         [0.],
         [0.],
         [1.]],

        [[0.],
         [1.],
         [0.],
         [0.],
         [0.]],


In [130]:
where_different.shape

torch.Size([50, 5, 5])

In [132]:
exps = differences.cuda().exp() * where_different.cuda()

In [133]:
exps

tensor([[[0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
         [3.2206e-03, 0.0000e+00, 7.4500e-04, 1.8580e-03, 1.6325e-03],
         [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00]],

        [[0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
         [1.1685e-04, 0.0000e+00, 4.2018e-04, 2.2793e-04, 3.9370e-05],
         [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00]],

        [[0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
         [6.5449e-05, 0.0000e+00, 1.0695e-04, 2.7849e-05, 5.8303e-05],
         [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
  

In [134]:
lsep = torch.log(1 + exps.sum(2).sum(1))

In [136]:
lsep.shape

torch.Size([50])

In [137]:
lsep.mean()

tensor(0.6654, device='cuda:0', grad_fn=<MeanBackward0>)

In [53]:
label_smoothing = 0.1
labels_smo = labels * (1 - label_smoothing) + 0.5 * label_smoothing

In [70]:
labels_smo

tensor([[0.0500, 0.9500, 0.0500, 0.0500, 0.0500],
        [0.0500, 0.9500, 0.0500, 0.0500, 0.0500],
        [0.0500, 0.9500, 0.0500, 0.0500, 0.0500],
        [0.0500, 0.9500, 0.0500, 0.0500, 0.0500],
        [0.9500, 0.0500, 0.0500, 0.0500, 0.9500],
        [0.9500, 0.0500, 0.0500, 0.0500, 0.9500],
        [0.0500, 0.9500, 0.0500, 0.0500, 0.0500],
        [0.9500, 0.0500, 0.0500, 0.0500, 0.9500],
        [0.9500, 0.0500, 0.0500, 0.0500, 0.9500],
        [0.0500, 0.9500, 0.0500, 0.0500, 0.0500],
        [0.9500, 0.0500, 0.0500, 0.0500, 0.9500],
        [0.9500, 0.0500, 0.0500, 0.0500, 0.9500],
        [0.0500, 0.9500, 0.0500, 0.0500, 0.0500],
        [0.9500, 0.0500, 0.0500, 0.9500, 0.0500],
        [0.0500, 0.9500, 0.0500, 0.0500, 0.0500],
        [0.9500, 0.0500, 0.9500, 0.0500, 0.0500],
        [0.0500, 0.9500, 0.0500, 0.0500, 0.0500],
        [0.9500, 0.0500, 0.0500, 0.0500, 0.0500],
        [0.9500, 0.0500, 0.0500, 0.0500, 0.0500],
        [0.0500, 0.9500, 0.0500, 0.0500, 0.0500],


In [ ]:
from predictions import *

In [ ]:
resnet

In [ ]:
from skorch.net import NeuralNet

In [ ]:
net = NeuralNet(
    resnet.module,
    max_epochs=20,
    verbose=0,
    criterion=nn.BCEWithLogitsLoss(),
    device='cuda',
    iterator_train__shuffle=True,
    iterator_train__num_workers=4,
)

In [ ]:
net

In [ ]:
valid_dataset[0]

In [ ]:
net.fit(valid_dataset, y=None);

In [ ]:
from skmultilearn.problem_transform import BinaryRelevance

In [ ]:
clf = BinaryRelevance(classifier=net, require_dense=[True,True])

In [ ]:
clf

In [ ]:
clf.fit(valid_loader.dataset)

In [ ]:
y_train

In [ ]:
labels = [y for x, y in iter(valid_loader.dataset)]

In [ ]:
labels

In [ ]:
torch.stack(labels)

In [ ]:
y_train = np.array(torch.stack(labels))

In [ ]:
x_train = valid_loader.dataset

In [ ]:
clf.fit(images, labels)

In [ ]:
len(y_train)

In [ ]:
len(x_train)

In [ ]:
len(valid_loader.dataset)

In [ ]:
labels = [y for x, y in iter(valid_loader.dataset)]

In [ ]:
images = [x for x, y in iter(valid_loader.dataset)]

In [ ]:
labels

In [ ]:
y_train = np.array(torch.cat(labels))

In [ ]:
len(y_train)

In [ ]:
images = 

In [ ]:
len(labels), len(images)

In [ ]:
torch.stack(labels)

In [ ]:
type(valid_loader.dataset)